In [169]:
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostRegressor
from algolab import API
from datetime import datetime
from config import *
import pandas as pd, numpy as np, json, os

import yfinance as yf

In [170]:
son=pd.read_excel("./excel/FiyatPenceresi.Xlsx")
endeks=son.iloc[-1,:]
son=son.iloc[:-1,:]
hisseler=son["Kod"].values
hisseler=np.array(hisseler)
for i in range(len(hisseler)):
    hisseler[i]=hisseler[i]+".IS"

In [171]:
df=pd.DataFrame()
for hisse in hisseler:
    hisse_verileri = yf.download(hisse,start="2010-01-01")  # Örnek tarih aralığı
    hisse_kapanis = hisse_verileri[['High','Low','Open','Close','Volume']]
    hisse_kapanis["Hisse"]=hisse
    hisse_kapanis.loc[hisse_kapanis["Close"] > hisse_kapanis["High"], "High"] = hisse_kapanis["Close"]
    hisse_kapanis.loc[hisse_kapanis["Close"] < hisse_kapanis["Low"], "Low"] = hisse_kapanis["Close"]

    df=pd.concat([df,hisse_kapanis],axis=0)
df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

In [ ]:
df["Volume"]=df["Volume"]*df["Close"]

In [ ]:
data=df.copy()

In [ ]:
df["Close"]=np.round(df["Close"],2)
df["High"]=np.round(df["High"],2)
df["Low"]=np.round(df["Low"],2)
df["Open"]=np.round(df["Open"],2)

In [ ]:
df.columns=['High', 'Low', 'Open', 'Adj Close', 'Volume', 'Hisse']

In [ ]:
canlı=pd.read_excel("./excel/FiyatPenceresi.Xlsx")
endekss=canlı.iloc[-1,:]
canlı=canlı.iloc[:-1,:]
canlı=canlı[["Yks","Dşk","Açl.Gün","Son.Fyt","Hcm","Kod","Lot"]]
canlı.columns=['High', 'Low', 'Open', 'Adj Close', 'Volume', 'Hisse',"Lot"]
canlı.index = pd.to_datetime(['2024-06-11'] * len(canlı))
canlı = canlı[~canlı['Hisse'].isin(['IDEAS', 'ISATR', 'ISKUR', 'ITTFH'])]
canlı

,High,Low,Open,Adj Close,Volume,Hisse,Lot
2024-06-11,"27,44","26,50","26,62","26,90",28.695.295,A1CAP,1.066.151
2024-06-11,"133,10","128,40","130,00","131,10",15.019.411,ACSEL,114.627
2024-06-11,"495,75","466,50","491,00","480,00",94.721.424,ADEL,195.851
2024-06-11,"2,26","2,17","2,19","2,18",279.280.770,ADESE,126.155.710
2024-06-11,"27,40","26,00","26,98","26,78",52.535.460,ADGYO,1.959.658
...,...,...,...,...,...,...,...
2024-06-11,"6,24","5,99","6,09","6,03",14.974.881,YYAPI,2.458.419
2024-06-11,"11,84","11,53","11,74","11,56",49.857.576,YYLGD,4.275.347
2024-06-11,"13,00","11,29","12,80","11,41",162.211.967,ZEDUR,13.535.534
2024-06-11,"5,36","5,22","5,26","5,25",443.662.662,ZOREN,83.852.512


In [ ]:
canlı["Lot"]=canlı["Lot"].astype("string")

In [ ]:
canlı=canlı.dropna()

In [ ]:
for i in range(len(canlı)):
    if len(canlı.iloc[i, -1]) <= 7:
        if '.' in canlı.iloc[i, -1]:
            parts = canlı.iloc[i, -1].split('.')
            decimal_part = parts[1]
            if len(decimal_part) == 1:
                decimal_part += '00'
            elif len(decimal_part) == 2:
                decimal_part += '0'
            elif len(decimal_part) == 3:
                canlı.iloc[i, -1] = parts[0] + decimal_part[:2]
            canlı.iloc[i, -1] = parts[0] + decimal_part


In [ ]:
kod=canlı["Hisse"].unique()
for i in range(len(kod)):
    kod[i]=kod[i]+".IS"

In [ ]:
fark=np.setdiff1d(df["Hisse"].unique(),kod)

In [ ]:
canlı["Hisse"]=kod

C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3390902008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canlı["Hisse"]=kod


In [ ]:
df = df[~df['Hisse'].isin(fark)]

In [ ]:
canlı["High"]=canlı["High"].str.replace(",",".")
canlı["Low"]=canlı["Low"].str.replace(",",".")
canlı["Open"]=canlı["Open"].str.replace(",",".")
canlı["Adj Close"]=canlı["Adj Close"].str.replace(",",".")
canlı["Lot"]=canlı["Lot"].astype(str).str.replace(".", "")
canlı[["High","Low","Open","Adj Close","Lot"]]=canlı[["High","Low","Open","Adj Close","Lot"]].astype(float)
canlı["Volume"]=canlı["Lot"]*canlı["Adj Close"]
canlı

C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3728428630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canlı["High"]=canlı["High"].str.replace(",",".")
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3728428630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canlı["Low"]=canlı["Low"].str.replace(",",".")
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3728428630.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,High,Low,Open,Adj Close,Volume,Hisse,Lot
2024-06-11,27.44,26.50,26.62,26.90,2.867946e+07,A1CAP.IS,1066151.0
2024-06-11,133.10,128.40,130.00,131.10,1.502760e+07,ACSEL.IS,114627.0
2024-06-11,495.75,466.50,491.00,480.00,9.400848e+07,ADEL.IS,195851.0
2024-06-11,2.26,2.17,2.19,2.18,2.750194e+08,ADESE.IS,126155710.0
2024-06-11,27.40,26.00,26.98,26.78,5.247964e+07,ADGYO.IS,1959658.0
...,...,...,...,...,...,...,...
2024-06-11,6.24,5.99,6.09,6.03,1.482427e+07,YYAPI.IS,2458419.0
2024-06-11,11.84,11.53,11.74,11.56,4.942301e+07,YYLGD.IS,4275347.0
2024-06-11,13.00,11.29,12.80,11.41,1.544404e+08,ZEDUR.IS,13535534.0
2024-06-11,5.36,5.22,5.26,5.25,4.402257e+08,ZOREN.IS,83852512.0


In [ ]:
canlı=canlı[canlı["Hisse"].isin(df["Hisse"].loc["2024-06-11"])]

In [ ]:
df["High"].loc["2024-06-11"]=canlı["High"].values
df["Low"].loc["2024-06-11"]=canlı["Low"].values
df["Open"].loc["2024-06-11"]=canlı["Open"].values
df["Adj Close"].loc["2024-06-11"]=canlı["Adj Close"].values
df["Volume"].loc["2024-06-11"]=canlı["Volume"].values

C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3418854252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["High"].loc["2024-06-11"]=canlı["High"].values
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3418854252.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Low"].loc["2024-06-11"]=canlı["Low"].values
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3418854252.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["O

In [ ]:
df["Kademe"] = np.where(df["Adj Close"] < 20, 0.01,
                np.where(df["Adj Close"] < 50, 0.02,
                    np.where(df["Adj Close"] < 100, 0.05,
                        np.where(df["Adj Close"] < 250, 0.1,
                            np.where(df["Adj Close"] < 500, 0.25,
                                np.where(df["Adj Close"] < 1000, 0.5,
                                    np.where(df["Adj Close"] < 2500, 1, 2.5)
                                )  # Buradaki parantez eksikti.
                            )
                        )
                    )
                )
            )


C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\1136949090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Kademe"] = np.where(df["Adj Close"] < 20, 0.01,


In [ ]:
df["Tavan"]=((((df["Adj Close"].shift(1))*1.1)/df["Kademe"]).fillna(0).astype(int))*df["Kademe"]
df["Tavan"]=np.round(df["Tavan"],2)

C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\1486124729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Tavan"]=((((df["Adj Close"].shift(1))*1.1)/df["Kademe"]).fillna(0).astype(int))*df["Kademe"]
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\1486124729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Tavan"]=np.round(df["Tavan"],2)


In [ ]:
df["Tavan Kontrol"]=np.where((df["Tavan"]==df["Adj Close"]),1,0)

C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\1300904211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Tavan Kontrol"]=np.where((df["Tavan"]==df["Adj Close"]),1,0)


In [ ]:
import pandas_ta as ta

df["SMA 5"]=df["Adj Close"].rolling(5).mean()
df["SMA 10"]=df["Adj Close"].rolling(5).mean()
df["SMA 200"]=df["Adj Close"].rolling(5).mean()

# RSI
df["RSI"] = ta.rsi(df["Adj Close"],14)

# Bağıl Hacim
df["Bağıl Hacim"] = df["Volume"] / df["Volume"].rolling(10).mean()



# ADX, DMI, Aroon
adx_data = ta.adx(df["High"], df["Low"], df["Adj Close"], 14)
df["ADX"], df["DMIP"], df["DMIN"] = adx_data.iloc[:, 0], adx_data.iloc[:, 1], adx_data.iloc[:, 2]




def calculate_stoch_rsi(data, rsi_period=14, stochastic_period=14, k_period=3, d_period=3):
    rsi = ta.rsi(data["Adj Close"], rsi_period)
    stoch_rsi = (rsi - rsi.rolling(window=stochastic_period).min()) / (rsi.rolling(window=stochastic_period).max() - rsi.rolling(window=stochastic_period).min())
    stoch_rsi_k = stoch_rsi.rolling(window=k_period).mean() * 100
    stoch_rsi_d = stoch_rsi_k.rolling(window=d_period).mean()
    data['StochRSI_%K'] = stoch_rsi_k
    data['StochRSI_%D'] = stoch_rsi_d
    return data

df = calculate_stoch_rsi(df)


def calculate_macd(data, short_period=12, long_period=26, signal_period=9):
    data['EMA_12'] = ta.ema(data['Adj Close'], short_period)
    data['EMA_26'] = ta.ema(data['Adj Close'], long_period)
    data['MACD'] = data['EMA_12'] - data['EMA_26']
    data['Signal_Line'] = ta.ema(data['MACD'], signal_period)
    data['MACD_above_Signal'] = (data['MACD'] > data['Signal_Line']).astype(float)
    return data

df = calculate_macd(df)



# Diğer Göstergeler
df["DD"] = ((df["Adj Close"] / df["High"]) - 1) * 100
df["Range"] = ((df["High"] - df["Low"]) / df["Adj Close"]) * 100

df["MOM"] = ta.mom(df["Adj Close"], 10)



df["Driehaus Momentum"] = ((df["Bağıl Hacim"] > 2) & (df["RSI"] > 50) & (df["MOM"] > 1) & (df["Adj Close"] > df["SMA 5"]) & (df["Adj Close"] > df["SMA 10"]) & (df["Adj Close"] > df["SMA 200"])).astype(int)
df["MACD Yukarı Kesen"] = ((df["Bağıl Hacim"] > 1.5) & (df["MACD"] > 0) & (df["MACD_above_Signal"] == 1)).astype(int)
df["ADXDMI20"] = ((df["Bağıl Hacim"] > 1.3) & (df["ADX"] > 20) & (df["DMIP"] > df["DMIN"])).astype(int)
df["MACD+Stoch"] = ((df["Bağıl Hacim"] > 1.3) & (df["StochRSI_%K"] > df["StochRSI_%D"]) & (df["MACD_above_Signal"] == 1)).astype(int)




C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\1098761594.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SMA 5"]=df["Adj Close"].rolling(5).mean()
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\1098761594.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SMA 10"]=df["Adj Close"].rolling(5).mean()
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\1098761594.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [ ]:
import pandas_ta as ta
import ta as ta1

df['MACD_Signal'] = df["Signal_Line"]
df['MACD_Diff'] = ta1.trend.macd_diff(df['Adj Close'])
df['ROC'] = ta.roc(df["Adj Close"],12)



C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3521117318.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_Signal'] = df["Signal_Line"]
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3521117318.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_Diff'] = ta1.trend.macd_diff(df['Adj Close'])
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3521117318.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [ ]:
df["Return"]=100*((df["Open"]/df["Adj Close"].shift(2))-1)
df["Return"]=df["Return"].shift(-2)
df["Date"]=pd.to_datetime(df.index)
df


C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\1013653962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Return"]=100*((df["Open"]/df["Adj Close"].shift(2))-1)
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\1013653962.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Return"]=df["Return"].shift(-2)
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\1013653962.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,High,Low,Open,Adj Close,Volume,Hisse,Kademe,Tavan,Tavan Kontrol,SMA 5,...,MOM,Driehaus Momentum,MACD Yukarı Kesen,ADXDMI20,MACD+Stoch,MACD_Signal,MACD_Diff,ROC,Return,Date
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-15,27.50,27.50,27.50,27.50,7.210885e+06,A1CAP.IS,0.02,0.00,0,NaN,...,NaN,0,0,0,0,NaN,NaN,NaN,12.727273,2023-06-15
2023-06-16,30.24,30.24,30.24,30.24,3.536565e+07,A1CAP.IS,0.02,30.24,1,NaN,...,NaN,0,0,0,0,NaN,NaN,NaN,-10.846561,2023-06-16
2023-06-20,31.10,29.94,31.00,29.94,2.414492e+08,A1CAP.IS,0.02,33.26,0,NaN,...,NaN,0,0,0,0,NaN,NaN,NaN,-14.428858,2023-06-20
2023-06-21,26.96,26.96,26.96,26.96,5.789431e+07,A1CAP.IS,0.02,32.92,0,NaN,...,NaN,0,0,0,0,NaN,NaN,NaN,-3.041543,2023-06-21
2023-06-22,27.62,25.50,25.62,25.94,1.865039e+09,A1CAP.IS,0.02,29.64,0,28.116,...,NaN,0,0,0,0,NaN,NaN,NaN,-0.154202,2023-06-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-05,8.07,7.67,7.97,7.69,1.789083e+07,ZRGYO.IS,0.01,8.74,0,7.930,...,0.12,0,0,0,0,0.195075,0.035199,0.918635,-1.170351,2024-06-05
2024-06-06,7.76,7.53,7.69,7.54,1.247644e+07,ZRGYO.IS,0.01,8.45,0,7.860,...,0.32,0,0,0,0,0.197202,0.008509,-1.694915,1.458886,2024-06-06
2024-06-07,8.08,7.50,7.60,7.57,9.847674e+07,ZRGYO.IS,0.01,8.29,0,7.742,...,0.47,0,0,1,0,0.195065,-0.008549,0.000000,-3.963012,2024-06-07


In [ ]:
endeks1=yf.download(tickers="XU100.IS",end="2020-07-27")/100
endeks2=yf.download(tickers="XU100.IS",start="2020-07-27")
endeks=pd.concat([endeks1,endeks2],axis=0)
df["Endeks"]=endeks["Adj Close"]
df["Endeks Return"]=100*((df["Endeks"]/df["Endeks"].shift(1))-1)
df["Endeks Return Lag"]=100*((df["Endeks"].shift(1)/df["Endeks"].shift(2))-1)
df["Return Lag"]=((df["Adj Close"]/df["Adj Close"].shift(1))-1)*100
df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3839789727.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Endeks"]=endeks["Adj Close"]
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3839789727.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Endeks Return"]=100*((df["Endeks"]/df["Endeks"].shift(1))-1)
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\383

,High,Low,Open,Adj Close,Volume,Hisse,Kademe,Tavan,Tavan Kontrol,SMA 5,...,MACD+Stoch,MACD_Signal,MACD_Diff,ROC,Return,Date,Endeks,Endeks Return,Endeks Return Lag,Return Lag
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-15,27.50,27.50,27.50,27.50,7.210885e+06,A1CAP.IS,0.02,0.00,0,NaN,...,0,NaN,NaN,NaN,12.727273,2023-06-15,5495.500000,NaN,NaN,NaN
2023-06-16,30.24,30.24,30.24,30.24,3.536565e+07,A1CAP.IS,0.02,30.24,1,NaN,...,0,NaN,NaN,NaN,-10.846561,2023-06-16,5475.500000,-0.363934,NaN,9.963636
2023-06-20,31.10,29.94,31.00,29.94,2.414492e+08,A1CAP.IS,0.02,33.26,0,NaN,...,0,NaN,NaN,NaN,-14.428858,2023-06-20,5251.600098,-4.089122,-0.363934,-0.992063
2023-06-21,26.96,26.96,26.96,26.96,5.789431e+07,A1CAP.IS,0.02,32.92,0,NaN,...,0,NaN,NaN,NaN,-3.041543,2023-06-21,5186.799805,-1.233915,-4.089122,-9.953240
2023-06-22,27.62,25.50,25.62,25.94,1.865039e+09,A1CAP.IS,0.02,29.64,0,28.116,...,0,NaN,NaN,NaN,-0.154202,2023-06-22,5428.299805,4.656050,-1.233915,-3.783383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-05,8.07,7.67,7.97,7.69,1.789083e+07,ZRGYO.IS,0.01,8.74,0,7.930,...,0,0.195075,0.035199,0.918635,-1.170351,2024-06-05,10351.000000,-0.131212,-1.820629,-3.270440
2024-06-06,7.76,7.53,7.69,7.54,1.247644e+07,ZRGYO.IS,0.01,8.45,0,7.860,...,0,0.197202,0.008509,-1.694915,1.458886,2024-06-06,10279.700195,-0.688820,-0.131212,-1.950585
2024-06-07,8.08,7.50,7.60,7.57,9.847674e+07,ZRGYO.IS,0.01,8.29,0,7.742,...,0,0.195065,-0.008549,0.000000,-3.963012,2024-06-07,10139.000000,-1.368719,-0.688820,0.397878


In [ ]:
endeks=endekss["Son.Fyt"]
endeks=endeks.replace(",",".")
endeks=float(endeks)

In [ ]:
index=yf.download(tickers="XU100.IS",start="2000-01-01")
endeks_son=((endeks/index["Adj Close"].iloc[-2])-1)*100
df["Endeks Return"].loc["2024-06-11"]=endeks_son
df


[*********************100%%**********************]  1 of 1 completed
C:\Users\Bora\AppData\Local\Temp\ipykernel_14920\3001027537.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Endeks Return"].loc["2024-06-11"]=endeks_son


,High,Low,Open,Adj Close,Volume,Hisse,Kademe,Tavan,Tavan Kontrol,SMA 5,...,MACD+Stoch,MACD_Signal,MACD_Diff,ROC,Return,Date,Endeks,Endeks Return,Endeks Return Lag,Return Lag
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-15,27.50,27.50,27.50,27.50,7.210885e+06,A1CAP.IS,0.02,0.00,0,NaN,...,0,NaN,NaN,NaN,12.727273,2023-06-15,5495.500000,NaN,NaN,NaN
2023-06-16,30.24,30.24,30.24,30.24,3.536565e+07,A1CAP.IS,0.02,30.24,1,NaN,...,0,NaN,NaN,NaN,-10.846561,2023-06-16,5475.500000,-0.363934,NaN,9.963636
2023-06-20,31.10,29.94,31.00,29.94,2.414492e+08,A1CAP.IS,0.02,33.26,0,NaN,...,0,NaN,NaN,NaN,-14.428858,2023-06-20,5251.600098,-4.089122,-0.363934,-0.992063
2023-06-21,26.96,26.96,26.96,26.96,5.789431e+07,A1CAP.IS,0.02,32.92,0,NaN,...,0,NaN,NaN,NaN,-3.041543,2023-06-21,5186.799805,-1.233915,-4.089122,-9.953240
2023-06-22,27.62,25.50,25.62,25.94,1.865039e+09,A1CAP.IS,0.02,29.64,0,28.116,...,0,NaN,NaN,NaN,-0.154202,2023-06-22,5428.299805,4.656050,-1.233915,-3.783383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-05,8.07,7.67,7.97,7.69,1.789083e+07,ZRGYO.IS,0.01,8.74,0,7.930,...,0,0.195075,0.035199,0.918635,-1.170351,2024-06-05,10351.000000,-0.131212,-1.820629,-3.270440
2024-06-06,7.76,7.53,7.69,7.54,1.247644e+07,ZRGYO.IS,0.01,8.45,0,7.860,...,0,0.197202,0.008509,-1.694915,1.458886,2024-06-06,10279.700195,-0.688820,-0.131212,-1.950585
2024-06-07,8.08,7.50,7.60,7.57,9.847674e+07,ZRGYO.IS,0.01,8.29,0,7.742,...,0,0.195065,-0.008549,0.000000,-3.963012,2024-06-07,10139.000000,-1.368719,-0.688820,0.397878


In [ ]:
df=df.sort_index()

In [ ]:
target_data = {
    "2024-02-13": "PATEK.IS",
    "2024-02-15": "BORSK.IS",
    "2024-02-22": "LMKDC.IS",
    "2024-02-29": "ALVES.IS",
    "2024-03-04": "ARTMS.IS",
    "2024-03-05": "MOGAN.IS",
    "2024-03-11": ["BARMA.IS", "INVES.IS", "EDATA.IS"],
    "2024-03-21": "ODINE.IS",
    "2024-04-26": "RGYAS.IS",
    "2024-05-02": ["OBAMS.IS", "ENTRA.IS"],
    "2024-05-09": "LILAK.IS",
    "2024-05-10": "KOTON.IS",
    "2024-05-16": "ALTNY.IS",
    "2024-05-17": "KOCMT.IS",
    "2024-05-23": "HRKET.IS",
    "2024-05-27": "PEHOL.IS",
    "2024-05-28": "ONRYT.IS",
    "2024-05-29": "OZYSR.IS",
    "2024-06-04": "ALKLC.IS",
    "2024-06-06": "YIGIT.IS",
    "2024-06-07": "HOROZ.IS"
}
for date, stocks in target_data.items():
    if isinstance(stocks, list):
        for stock in stocks:
            df.loc[(df.index == date) & (df["Hisse"] == stock), "Tavan Kontrol"] = 1
    else:
        df.loc[(df.index == date) & (df["Hisse"] == stocks), "Tavan Kontrol"] = 1
son=df.loc["2024-06-10":]
ilk=df.loc[:"2024-06-07"]
ilk=ilk[(ilk["Return"]<21) & (ilk["Return"]>-19)]
result=pd.concat([ilk,son],axis=0)

In [ ]:
df=df.sort_index()

yasak=["YYAPI","EMNIS","TETMT","RODRG","BRKO","AYES","EUKYO","YGYO","BALAT","SONME","SNKRN","KSTUR","DERIM","UZERB","MARKA","MMCAS","YAYLA"]
for i in range(len(yasak)):
    yasak[i]=yasak[i]+".IS"
result=result.dropna()
result = result[~result['Hisse'].isin(["ISATR.IS","ISBTR.IS"])]
result = result[~result['Hisse'].isin(yasak)]
result=result[result["Tavan Kontrol"]==0]
result=result.sort_index()


In [ ]:

test=result.loc["2024-06-11"]
X_test=test[['Endeks Return','Return Lag','Range','RSI','Volume','Bağıl Hacim','DD','High', 'Low', 'Open', 'Adj Close','StochRSI_%K','MACD','ROC','MACD_Signal','MOM']]


In [ ]:
params = {'iterations': 406,
 'depth': 8,
 'learning_rate': 0.05908481614373525,
 'random_strength': 15,
 'bagging_temperature': 0.48897432777080385,
 'border_count': 163,
 'l2_leaf_reg': 9}
model=CatBoostRegressor(**params,task_type="GPU",random_seed=42)
model.load_model("./model/xtumyson.bin")

In [ ]:
tahminler=pd.DataFrame(model.predict(X_test))
tahminler=tahminler.set_index(X_test.index)
tahminler.columns=["Tahmin"]
tahminler["Hisse"]=result["Hisse"].loc["2024-06-11"]
tahminler["Tavan"]=result["Tavan Kontrol"].loc["2024-06-11"]
tahminler=tahminler[tahminler["Tavan"]==0]
tahminler=tahminler.sort_values(by="Tahmin",ascending=False).head(3)
tahminler

,Tahmin,Hisse,Tavan
Date,,,
2024-06-11,3.992080,ANHYT.IS,0
2024-06-11,1.615364,DEVA.IS,0
2024-06-11,1.586995,BSOKE.IS,0


In [ ]:
bakiye=2209
veri=tahminler
veri=veri.sort_values(by="Hisse")
veri["Fiyat"]=df[df["Hisse"].isin(veri["Hisse"])].loc["2024-06-11"].sort_values(by="Hisse")["Adj Close"].values
veri["Para"]=bakiye*0.33
veri["Lot"]=veri["Para"]//(veri["Fiyat"])
veri

ValueError: Length of values (520) does not match length of index (53283)

In [ ]:
hisseler=veri["Hisse"].values
for i in range(len(hisseler)):
    hisseler[i]=hisseler[i][:-3]
hisseler

array(['ANHYT', 'BSOKE', 'DEVA'], dtype=object)

In [ ]:
hisseler_str = ", ".join(hisseler)

In [ ]:
import telegram

# Bot token'ınızı buraya yapıştırın
BOT_TOKEN = '6994011653:AAH8kaRmhMqfHyp634PiaTP1ri63onS56Nk'
# Kanal kullanıcı adı ya da ID'si
CHANNEL_ID = '6356198553'  # ya da '-1001234567890' formatında kanal ID'si

# Bot nesnesini oluşturun
bot = telegram.Bot(token=BOT_TOKEN)

# Gönderilecek mesaj
message = f"11.06.2024 kapanışta alınan hisseler:{hisseler_str}"
message1=f"Hisseler 13.06.2024 tarihinde piyasa açılışında satılacaktır."
# Mesajı gönderin
bot.send_message(chat_id=CHANNEL_ID, text=message)
bot.send_message(chat_id=CHANNEL_ID, text=message1)

In [ ]:
"""from algolab import API
from datetime import datetime
from config import *
import pandas as pd, numpy as np, json, os
Conn = API(api_key=MY_API_KEY, username=MY_USERNAME, password=MY_PASSWORD, auto_login=True, verbose=True)"""

'from algolab import API\nfrom datetime import datetime\nfrom config import *\nimport pandas as pd, numpy as np, json, os\nConn = API(api_key=MY_API_KEY, username=MY_USERNAME, password=MY_PASSWORD, auto_login=True, verbose=True)'

In [ ]:
"""symbols=veri["Hisse"].values
for i in range(len(symbols)):
    symbols[i]=symbols[i][:-3]
symbols"""

'symbols=veri["Hisse"].values\nfor i in range(len(symbols)):\n    symbols[i]=symbols[i][:-3]\nsymbols'

In [ ]:
"""for i in range(len(veri)):
   order = Conn.SendOrder(symbol=symbols[i], direction= "buy", pricetype= "limit", price=veri["Fiyat"].iloc[i], lot=str(veri["Lot"].iloc[i]) ,sms=True,email=False,subAccount="")
   print(order) """   
   

'for i in range(len(veri)):\n   order = Conn.SendOrder(symbol=symbols[i], direction= "buy", pricetype= "limit", price=veri["Fiyat"].iloc[i], lot=str(veri["Lot"].iloc[i]) ,sms=True,email=False,subAccount="")\n   print(order) '